In [ ]:
! pip install pymongo
! pip install dnspython
! pip install python-dotenv

In [1]:
from pymongo import MongoClient
import os
import dotenv


#### Connectivity

##### Google colab

In [ ]:
# # google colab access
# from google.colab import userdata
# mUser = userdata.get('optimus_mongodb_all_user')
# mPwd = userdata.get('optimus_mongodb_all_user_pwd')
# mCluster = userdata.get('optimus_mongodb_cluster')

##### OS environ

In [ ]:
# os environ
dotenv.load_dotenv(os.path.join(os.path.dirname(os.getcwd()), "env", "optimus_mongo.env"))
mUser = os.environ['optimus_mongodb_all_user']
mPwd = os.environ['optimus_mongodb_all_user_pwd']
mCluster = os.environ['optimus_mongodb_cluster']

In [ ]:
connection_url = f"mongodb+srv://{mUser}:{mPwd}@{mCluster}"

In [ ]:
mongo_client = MongoClient(connection_url) 

In [ ]:
mongo_db = mongo_client['groceries_dataset']

In [ ]:
customersColl = mongo_db['customers']
employeesColl = mongo_db['employees']
salesColl = mongo_db['sales']
productsColl = mongo_db['products']


### Customers Collection

#### Fetching sample record from customers collection - using find

In [ ]:
resultSet = []

cursorSet = customersColl.find({})

for item in cursorSet:
  resultSet.append(item)
  break

In [ ]:
resultSet

#### Fetching sample record from customers collection - using match
aggreate -> match gives us control over modifying the result set

In [ ]:
customersColl.aggregate([{'$match':{"CustomerID":17}}]).to_list()

#### Get the total count of customer status group by values

In [ ]:
# getting count of customers basis upon city
customersColl.aggregate([
    {'$group': {
    '_id':'$CityName',    # if we do not keep $ here, then it will give us total count of customers for all the cities together
    'totalCount': {'$count': {}}
}},

   {'$sort': {"_id": -1}},
                         {'$limit':5}]).to_list()

In [ ]:
# getting count of customers basis upon city -
customersColl.aggregate([{'$group': {
    '_id':'$CityName',    # if we do not keep $ here, then it will give us total count of customers for all the cities together
    'totalCount': {'$count': {}}
}},

   {'$sort': {"_id": 1}},
                           {'$limit':5}]).to_list()

#### group by multiple columns

In [ ]:
customersColl.aggregate([
    {'$group':{ '_id': {
        '_CityName': '$CityName',
        '_CountryName' : '$CountryName'
    },
     'totalCustomersCount':{'$count': {}}}},
    {'$sort': {
        '_id._CityName' : 1
    }},
    {'$limit': 5}
]).to_list()

### Employees Collection

#### Sample Record

In [ ]:
print(employeesColl.count_documents({}))
employeesColl.find({}).limit(1)[0]

### Products Collection

#### Sample record

In [ ]:
print(productsColl.count_documents({}))
productsColl.find({}).limit(1)[0]

#### group the products with class

In [ ]:
productsColl.aggregate([
    {'$group': {
        '_id':'$Class',
        'totalProductsCount': {'$count':{}}
    }
}]).to_list()

#### find the costly item in products

In [ ]:
productsColl.find({}).sort("Price", -1).limit(1).to_list() # taking first sample doc

### Sales Collection

#### sample data

In [ ]:
print(salesColl.count_documents({}))
salesColl.find({}).limit(1)[0]

#### month wise sales count

In [ ]:
salesColl.aggregate([
    # first we have to project the month, basically preparing the data for group
    {
        '$project': {
            'month' : {'$month': {'$toDate': '$SalesDate'}}, # need to convert this into date format
            'year' : {'$year': {'$toDate': '$SalesDate'}} # need to convert this into date format
        }
    },
    {
    '$group': {
        '_id': {
            '_year':'$year',
            '_month':'$month'
        },
        'totalSalesCount': {'$count': {}}
      }
    },
    {
        '$sort': {"totalSalesCount": -1}
    }
    ]).to_list()

#### get the SalesPersonID who has max sales

In [ ]:
salesColl.aggregate([
    {
        '$group': {
            '_id': '$SalesPersonID' ,
            'totalSalesCount' : {'$count' : {}}
        }
    },
    {
        '$sort': { 'totalSalesCount' : -1}
    }
]).to_list()

### Combined Queries

#### Get the name of the employee who has max sales count

#### Total Count of orders for each product category

In [ ]:
[
  {
    $lookup: {
      from: "employees",
      localField: "SalesPersonID",
      foreignField: "EmployeeID",
      pipeline: [
        {
          $project: {
            EmployeeName: {
              $concat: [
                "$FirstName",
                " ",
                "$MiddleInitial",
                " ",
                "$LastName",
              ],
            },
          },
        },
        {$group: {
          '_id' : "$SalesPersonID",
          "totalCount" : {$count : {}}
        }},
        { $limit: 1 },
      ],
      as: "EmployeeInfo",
    },
  },
]

In [ ]:
product_collection.aggregate([{
    "$lookup": {
        "from": "orders",
        "localField": "product_id",
        "foreignField": "product_id",
        "pipeline": [{
            "$group": {"_id": "order_id", "totalOrders": {"$count": {}}}
        }],
        "as":"totalOrdersInfo"
    }
}]).to_list()[0]

In [ ]:
[
  {
    $facet: {
      maxSalesCountEmpID: [
        {
          $group: {
            _id: "$SalesPersonID",
            totalSalesCount: { $count: {} },
          },
        },
        { $sort: { totalSalesCount: -1 } },
        {
          $project: {
            _id: 1,
          },
        },
      ],
      maxSalesCountEmpInfo: [
        {
          $lookup: {
            from: "employees",
            localField: "EmployeeID",
            foreignField: "SalesPersonID",
            pipeline: [
              {
                $match: {
                  EmployeeID: 5,
                },
              },
              {$limit :5},
              {$project: {
                'SalesPersonID':1,
                'EmployeeInfo':1
              }}
            ],
            as: "EmployeeInfo",
          },
        },
        {$limit :1},
      ],
    },
  },
  { $unwind: "$maxSalesCountEmpID"},
  {$limit :5}
]